In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torchvision.datasets import FashionMNIST,StanfordCars
from matplotlib import pyplot as plt
import numpy as np
import torch.nn.functional as F
import math
import pandas as pd
from PIL import Image

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
# Defining the device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return 
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [5]:
class Encoder(nn.Module):
    def __init__(self,block,layers,size):
        self.inplanes=64
        super(Encoder,self).__init__()
        self.conv1=nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.layer1=self._make_layer(block,64,layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.layer5 = self._make_layer(block, 1024, layers[4], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(49152, size) ## CHANGE FROM HERE 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        
        x=self.maxpool(x)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        
        x=self.avgpool(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        
        return x

In [6]:
# encoder = Encoder(Bottleneck, [3, 4, 6, 3,3,1])
# test=torch.ones((4,3,480,720))
# encoder(test).shape

In [7]:
class Decoder(nn.Module):
    def __init__(self,size):
        super(Decoder,self).__init__()
        self.dfc3=nn.Linear(size,4096)
        self.bn3=nn.BatchNorm1d(4096)
        self.dfc2=nn.Linear(4096,4096)
        self.bn2=nn.BatchNorm1d(4096)
        self.dfc1=nn.Linear(4096,256*6*6)
        self.bn1=nn.BatchNorm1d(256*6*6)
        self.upsample1=nn.Upsample(scale_factor=(2.648, 3))
#         self.unflatten1 = nn.Unflatten(1, ())
        self.dconv5 = nn.ConvTranspose2d(256, 256, 3, padding = 0)
        self.dconv4 = nn.ConvTranspose2d(256, 384, 3, padding = 1)
        self.dconv3 = nn.ConvTranspose2d(384, 192, 3, padding = 1)
        self.dconv2 = nn.ConvTranspose2d(192, 64, 5, padding = 2)
        self.dconv1 = nn.ConvTranspose2d(64, 3, 24, stride = 4, padding = (8,10))
    def forward(self,x):
        x = self.dfc3(x)
        x = F.relu(self.bn3(x))
        
        x = self.dfc2(x)
        x = F.relu(self.bn2(x))
        x = self.dfc1(x)
        x = F.relu(self.bn1(x))
        x = x.view(-1,256,6,6)
        x=self.upsample1(x)
        x = self.dconv5(x)
        x = F.relu(x)
        x = F.relu(self.dconv4(x))
        x = F.relu(self.dconv3(x))
        x=self.upsample1(x)
        x = self.dconv2(x)
        x = F.relu(x)
        x=self.upsample1(x)

        x = self.dconv1(x)

        x = torch.sigmoid(x)
        return x

In [ ]:
# tester=torch.ones((4,dims))
# decoder=Decoder(dims)
# (decoder(tester)).shape

In [8]:
class VAE(nn.Module):
    def __init__(self,dims):
        super(VAE,self).__init__()
        self.encoder = Encoder(Bottleneck, [3, 4, 6, 3,3,1],dims*2)
        self.decoder = Decoder(dims)
        self.dims=dims
    def reparameterise(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def forward(self,y):
        mu_logvar = self.encoder(y)
        mu_logvar=mu_logvar.view(-1,2,self.dims)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]
        z = self.reparameterise(mu, logvar).view(-1,self.dims)
        z=self.decoder(z)
        return z,mu,logvar
        

In [ ]:
# ?torch.nn.ReLU

In [ ]:
# # Defining the model

# d = 256

# class VAE(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.encoder1 = nn.Sequential(
#             nn.Conv2d(3,4,4),
#             nn.ReLU(),
#             nn.Conv2d(4,1,4),
#             nn.MaxPool2d(2),
#             nn.Flatten(),
#             nn.Linear(84609,d*2)
#         )

#         self.decoder1 = nn.Sequential(
#             nn.ConvTranspose2d(1,8,4,stride=2),
#             nn.ReLU(),
#             nn.ConvTranspose2d(8,4,4,stride=1,padding=1),
#             nn.ReLU(),
#             nn.ConvTranspose2d(4,4,4,stride=1),
#             nn.Sigmoid()
#         )
#         self.transfer=nn.Sequential(
#             nn.Linear(d,84609),
#             nn.Unflatten(1,(237,357))
#         )
        
        
#     def reparameterise(self, mu, logvar):
#         if self.training:
#             std = logvar.mul(0.5).exp_()
#             eps = std.data.new(std.size()).normal_()
#             return eps.mul(std).add_(mu)
#         else:
#             return mu

#     def forward(self, y):
#         mu_logvar = self.encoder1(y)
# #         print(mu_logvar.shape)
# #         z=0
# #         mu1=0
# #         logvar1=0
#         mu_logvar=mu_logvar.view(-1,2,d)
#         mu1= mu_logvar[:, 0, :]
#         logvar1 = mu_logvar[:, 1, :]
#         z = self.reparameterise(mu1, logvar1).view(-1,d)
# #         print(z.shape)
#         z = self.transfer(z).view(-1,1,237,357)
#         z=self.decoder1(z)
# #         print("output=",z.shape)
#         return z,mu1,logvar1

In [9]:
dims=1024
model = VAE(dims).to(device)

In [10]:

# Setting the optimiser

learning_rate = 1e-3

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [11]:
# Reconstruction + KL divergence losses summed over all elements and batch

def loss_function(ỹ, y, mu1, logvar1):
    BCE = nn.functional.binary_cross_entropy(
        ỹ, y, reduction='sum'
    )
    KLD = (-0.5 * torch.sum(-logvar1.exp() + logvar1 + 1.0 - mu1.pow(2)))
    return BCE + KLD,BCE,KLD

In [12]:
class MyDataset(Dataset):
    def __init__(self, train_path):
        self.df = pd.read_csv(train_path, sep=',', usecols=['input', 'output'])
#         print(self.df.shape)
    def __getitem__(self, index):
        x = np.array(Image.open(self.df.iloc[index, 1]))
        y = np.array(Image.open(self.df.iloc[index, 0]))
        x, y = torch.from_numpy(x), torch.from_numpy(y)
        return x, y

    def __len__(self):
        return 2000

In [13]:
train_loader=MyDataset("./dataset_train.csv")
train_loader=DataLoader(train_loader, batch_size=2,shuffle=True)


train_loader

In [14]:
# Training and testing the VAE
do=nn.Dropout()
epochs = 100
codes = dict(μ=list(), logσ2=list(), x=list())
for epoch in range(0, epochs + 1):
    # Training
    if epoch > 0:  # test untrained net first
        model.train()
        train_loss = 0
        bcs=0
        kls=0
        for x,y in train_loader:
            x = x.to(device)
            y = y.to(device)
            x=x.view(-1,3,480,720)
            y=y.view(-1,4,480,720)
            x=torch.div(x,255)
            y=torch.div(y,255)
            # ===================forward=====================
            y_bar, mu1, logvar1 = model(x)
            loss,bc,kl = loss_function(y_bar, y, mu1, logvar1)
            train_loss += loss
            bcs+=bc.item()
            kls+=kl
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
#             print(loss.item())
            optimizer.step()
        # ===================log========================
        print(f'====> Epoch: {epoch} Average loss: {train_loss /20000} BCE Loss: {bcs / 2000} KLD Loss: {kls / 2000}')
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, "./Weights/resnet.pt")
        
    # Testing
    
#     means, logvars, labels = list(), list(), list()
#     with torch.no_grad():
#         model.eval()
#         test_loss = 0
#         bcs=0
#         kls=0
#         for y, x in test_loader:
#             y = y.to(device)
#             y=y.view(-1,1,28,28)
#             # ===================forward=====================
#             ỹ, mu1, logvar1,mu2,logvar2,mu3,logvar3 = model(y)
#             # print(ỹ.shape)
#             # print(y.shape)
#             loss,bc,kl = loss_function(ỹ, y, mu1, logvar1,mu2,logvar2,mu3,logvar3)
#             test_loss+=loss.item()
#             bcs+=bc.item()
#             kls+=kl.item()
#             # =====================log=======================
#             means.append(mu3.detach())
#             logvars.append(logvar3.detach())
#             labels.append(x.detach())
#     # ===================log========================
#     codes['μ'].append(torch.cat(means))
#     codes['logσ2'].append(torch.cat(logvars))
#     codes['x'].append(torch.cat(labels))
#     test_loss /= len(test_loader.dataset)
#     print(f'Average loss: {test_loss} BCE Loss: {bcs / len(test_loader.dataset):.4f} KLD Loss: {kls / len(test_loader.dataset):.4f}')

ValueError: Using a target size (torch.Size([2, 4, 480, 720])) that is different to the input size (torch.Size([2, 3, 480, 720])) is deprecated. Please ensure they have the same size.

In [ ]:
torch.cuda.empty_cache()

In [ ]:
test_loader=MyDataset("./dataset_test.csv")

In [ ]:
temp =np.array(Image.open("./Datasets/Input/Echendens-LHS_09620.png_6.png"), dtype = float)/255.0

In [ ]:
tem = torch.from_numpy(temp).view(-1,3,480,720)

In [ ]:
tem=tem.to(device,dtype=torch.float32)

In [ ]:
ans=(model(tem))

In [ ]:
ans=(ans[0]*255).detach().cpu().numpy()

In [ ]:
ans.shape

In [ ]:
img=Image.fromarray(ans)